In [65]:
import pandas as pd

from utils import only_2016_data

spatial = pd.read_csv('data/grunnkrets_norway_stripped.csv')
age = pd.read_csv('data/grunnkrets_age_distribution.csv')
income = pd.read_csv('data/grunnkrets_income_households.csv').set_index(['grunnkrets_id', 'year']).add_prefix('income_').reset_index()
households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
submission = pd.read_csv('data/sample_submission.csv')
plaace = pd.read_csv('data/plaace_hierarchy.csv')
busstops = pd.read_csv('data/busstops_norway.csv')

train = pd.read_csv('data/stores_train.csv') #.drop(columns=['year'])
test = pd.read_csv('data/stores_test.csv') 

In [66]:
spatial_2016 = only_2016_data(spatial)
income_2016 = only_2016_data(income)
households_2016 = only_2016_data(households)

train_spatial = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_income = train.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_house = train.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

train_house_no_nan = train_house[pd.notnull(train_house.couple_children_0_to_5_years)]
train_spatial_no_nan = train_spatial[pd.notnull(train_spatial.grunnkrets_name)]
train_income_no_nan = train_income[pd.notnull(train_income.income_all_households)]

In [ ]:
def clean_out_nan_heavy_rows(df: pd.DataFrame):
    """Cleans out rows that have no match in the spatial, income or household datasets."""

    df2 = df.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
    df2 = df2.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
    df2 = df2.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

    df_cleaned = df2[
        ~(df2.couple_children_0_to_5_years.isna() | df2.grunnkrets_name.isna() | df2.income_all_households.isna())
    ]

    print(f'Cleaned out {len(df) - len(df_cleaned)} out of {len(df)} rows.')

    return df_cleaned



In [76]:
test_df = test.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
test_df = test_df.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
test_df = test_df.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

assert len(test) == len(test_df)

test_cleaned = test_df[
    ~(test_df.couple_children_0_to_5_years.isna() | test_df.grunnkrets_name.isna() | test_df.income_all_households.isna())
]


In [80]:
print(len(test), len(test_cleaned), len(test_df))
test.isna().sum()
test_cleaned.isna().sum()
test_df.isna().sum()

8577 8507 8577


store_id                                   0
year                                       0
store_name                                 0
plaace_hierarchy_id                        0
sales_channel_name                         0
grunnkrets_id                              0
address                                 1237
lat                                        0
lon                                        0
chain_name                              6099
mall_name                               7084
grunnkrets_name                           27
district_name                             27
municipality_name                         27
geometry                                  27
area_km2                                  27
income_all_households                     27
income_singles                            27
income_couple_without_children            27
income_couple_with_children               27
income_other_households                   27
income_single_parent_with_children        27
couple_chi

In [68]:
train_df = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_df = train_df.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_df = train_df.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

assert len(train) == len(train_df)

train = train_df[
    ~(train_df.couple_children_0_to_5_years.isna() | train_df.grunnkrets_name.isna() | train_df.income_all_households.isna())
]





AttributeError: 'DataFrame' object has no attribute 'couple_children_0_to_5_years'

In [70]:
print(len(train), len(train_df))
train.isna().sum()

12732 12732


store_id                                    0
year                                        0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1756
lat                                         0
lon                                         0
chain_name                               9030
mall_name                               10479
revenue                                     0
grunnkrets_name                             0
district_name                               0
municipality_name                           0
geometry                                    0
area_km2                                    0
income_all_households                       0
income_singles                              0
income_couple_without_children              0
income_couple_with_children                 0
income_other_households           

In [25]:
train_house_nans = train_house.grunnkrets_id[pd.isnull(train_house.couple_children_0_to_5_years)]
train_spatial_nans = train_spatial.grunnkrets_id[pd.isnull(train_spatial.grunnkrets_name)]
train_income_nans = train_income.income_all_households[pd.isnull(train_income.income_all_households)]

train_cleaned = train[
    ~(train.grunnkrets_id.isin(train_house_nans)) \
    | ~(train.grunnkrets_id.isin(train_spatial_nans)) \
    | ~(train.grunnkrets_id.isin(train_income_nans))
]

print(len(train), len(train_cleaned))
train_cleaned.isna().sum()

train.grunnkrets_id.isin(train_house_nans).value_counts()

12859 12859


False    12735
True       124
Name: grunnkrets_id, dtype: int64

In [6]:
train_full = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_full = train_full.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_full = train_full.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

train_full.isna().sum()

store_id                                    0
year                                        0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1774
lat                                         0
lon                                         0
chain_name                               9122
mall_name                               10579
revenue                                     0
grunnkrets_name                            30
district_name                              30
municipality_name                          30
geometry                                   30
area_km2                                   30
income_all_households                      33
income_singles                             33
income_couple_without_children             33
income_couple_with_children                33
income_other_households           